In [1]:
from huggingface_hub import login
import os
from dotenv import load_dotenv

load_dotenv()
hf_token = os.getenv("HUGGING_FACE_TOKEN")
login(token=hf_token)

In [2]:
from perplexity import Perplexity
from dotenv import load_dotenv

load_dotenv() 

client = Perplexity() # PERPLEXITY_API_KEY is read from environment variable

search = client.search.create(
    query=[
      "What is Comet Browser?",
      "Perplexity AI",
      "Perplexity Changelog"
    ]
)

for result in search.results:
    print(f"{result.title}: {result.url}")

Introducing Comet: Browse at the speed of thought - Perplexity: https://www.perplexity.ai/hub/blog/introducing-comet
Browse at the speed of thought: https://www.perplexity.ai/comet/
Comet Browser Review 2025: Features, Pricing, Pros & Cons: https://efficient.app/apps/comet
Perplexity's AI-Native Browser Comet is HERE: https://www.youtube.com/watch?v=ri_bFrDp44M
Comet Browser: The Revolutionary AI-Powered ...: https://dev.to/s_chathuranga_j/comet-browser-the-revolutionary-ai-powered-browser-that-actually-does-your-work-n6d
What Is Comet and Why Everyone's Talking About It: https://apidog.com/blog/what-is-comet/
10 Comet Browser Features That Make Chrome & Edge Look ANCIENT: https://www.youtube.com/watch?v=BfC_Lk3NVSs
Perplexity launches Comet AI browser for everyone: How to try it: https://mashable.com/article/perplexity-comet-ai-web-browser-how-to-try
Comet: The AI browser gets personal: https://www.ibm.com/think/news/comet-perplexity-take-agentic-browser
Perplexity Comet AI Browser 10

In [3]:
import json
from pprint import pprint

goal = "Analyze APPL stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create a detailed plan to achieve this goal:
    Goal: {goal}
    
    Available tools: web search, financial data API, news aggregator, sentiment analysis tool

    Create a plan with:
    1. Logical research steps (3-7 steps)
    2. Estimated time in minutes
    3. Required tools for each step
    4. Success criteria to evaluate completion
    
    Answer in JSON format:
    {{
        "goal": "...",
        "steps": ["step1", "step2", ...],
        "estimated_time": 30,
        "required_tools": ["tool1", "tool2", ...],
        "success_criteria": ["criteria1", "criteria2", ...]
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)


### Presenting the data generated

In [4]:

#print("Raw Response:\n", response.choices[0].message.content)
plan_data = json.loads(response.choices[0].message.content)

pprint(plan_data)

{'estimated_time': 45,
 'goal': 'Analyze APPL stock for a potential investment.',
 'required_tools': ['financial data API',
                    'web search',
                    'news aggregator',
                    'sentiment analysis tool'],
 'steps': ['Collect and analyze historical stock price data and forecasts '
           'using financial data API and research papers to understand price '
           'trends and model predictions (e.g., regression, smoothing, '
           'SARIMA).',
           'Perform a comprehensive fundamental analysis by reviewing Apple’s '
           'financial statements, including profitability, liquidity, '
           'solvency, and valuation ratios using financial data API and news '
           'aggregator.',
           'Evaluate intrinsic valuation metrics such as discounted cash flow '
           '(DCF), dividend discount model (DDM), beta, equity risk premium, '
           'and sensitivity analyses using financial data API and research '
           

### Sentiment analysis using perplexity API

#### Get information from FinTools

In [5]:
from src.ingestion import FinancialDataIngestion, NewsDataIngestion

ticker = "PLTR"  # Example stock ticker
stock_info = FinancialDataIngestion().execute(ticker)
pprint(stock_info)

news_articles = NewsDataIngestion().execute(ticker, 5)
for article in news_articles:
    pprint(article)


{'data': {'52_week_high': 190.0,
          '52_week_low': 40.36,
          'avg_volume_30d': 55596890.90909091,
          'beta': 2.595,
          'company_summary': 'Palantir Technologies Inc. builds and deploys '
                             'software platforms for the intelligence '
                             'community to assist in counterterrorism '
                             'investigations and operations in the United '
                             'States, the United Kingdom, and internationally. '
                             'It provides Palantir Gotham, a software '
                             'platform, which enables users to identify '
                             'patterns hidden deep within datasets, ranging '
                             'from signals intelligence sources to reports '
                             'from confidential informants, as well as '
                             'facilitates the hand-off between analysts and '
                             'op

#### Generate sentiment analysis and confidence score

In [6]:

goal = f"Analyze {ticker} stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create short sentiment analysis
      about the stock based on the context and recent news.
      Describe the sentiment as Positive, Negative, or Neutral.
      Give a summary of your analysis, and a confidence score between 0 and 1.
    Goal: {goal}

    Context: {stock_info}

    Recent News: {news_articles}

    Answer in JSON format:
    {{
        "stock": "...",
        "sentiment": "Positive",
        "summary": "...",
        "confidence": 0.85
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)

print("Raw Response:\n", response.choices[0].message.content)

Raw Response:
 ```json
{
    "stock": "PLTR",
    "sentiment": "Positive",
    "summary": "Palantir Technologies Inc. (NASDAQ:PLTR) has shown significant growth, with a surge of nearly 142% this year and a remarkable increase of 1,875.43% since its IPO in October 2022. The stock is currently near its buy point, suggesting potential for further growth. Additionally, prominent figures like Jim Cramer have expressed optimism about Palantir's prospects. The recent price increase and positive market positioning contribute to a positive sentiment. However, the absence of dividends and a high price-to-earnings ratio might introduce some caution for investors.",
    "confidence": 0.85
}
```


## Adding Memory to the planner operation as an example

### 1. add memories

In [7]:
from src.memory import MemoryWorker

# adding memories as data example
memory_worker = MemoryWorker()
memory_worker.execute('add', 'AAPL Q2 earnings beat expectations', ['AAPL', 'earnings'])
memory_worker.execute('add', 'AAPL announced a new AI partnership with NASA', ['AAPL', 'AI'])
memory_worker.execute('add', 'TSLA stock is volatile due to production concerns', ['TSLA', 'production'])

print("3 memories added.")

3 memories added.


### 2. ingest data

In [8]:
from src.memory import MemoryWorker


### Search Memories by tag

ticker = "AAPL"  # Example stock ticker
stock_info = FinancialDataIngestion().execute(ticker)
news_articles = NewsDataIngestion().execute(ticker, 5)
memories = memory_worker.execute('search', 'AAPL')


### Prompting for analysis

It demonstrates how the memory impacts the results. 
You can see that the added memories will appear on the summary

In [9]:

goal = f"Analyze {ticker} stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create short sentiment analysis
      about the stock based on the context and recent news.
      Describe the sentiment as Positive, Negative, or Neutral.
      Give a summary of your analysis, and a confidence score between 0 and 1.
    Goal: {goal}

    Context: {stock_info}

    Recent News: {news_articles}

    Relevant Memories: {memories}

    Answer in JSON format:
    {{
        "stock": "...",
        "sentiment": "Positive",
        "summary": "...",
        "confidence": 0.85
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)

print("Raw Response:\n", response.choices[0].message.content)

Raw Response:
 ```json
{
    "stock": "AAPL",
    "sentiment": "Positive",
    "summary": "Overall, Apple's stock sentiment remains positive due to its strong financial performances, innovative partnerships like the new AI partnership with NASA, and its dominant position in the consumer electronics sector. Recent news indicates some market volatility and price fluctuations, but the company's fundamentals and strategic moves continue to bolster investor confidence. The upcoming fiscal Q4 financials are expected to further solidify Apple's market standing.",
    "confidence": 0.8
}
```


### Ingestion worker

In [12]:
from src.ingestion import Ingestion

ingestion_worker = Ingestion()
ingestion_worker.execute('AAPL')

{'symbol': 'AAPL',
 'timestamp': '2025-10-10T04:24:28.399615',
 'financial_data': {'current_price': 254.0399932861328,
  'price_change': -4.02000671386719,
  'price_change_pct': -1.5577798627711346,
  'volume': 38270200,
  'avg_volume_30d': 55872113.63636363,
  'volatility_30d': 0.21924060897119982,
  'market_cap': 3770052509696,
  'pe_ratio': 30.570395,
  'dividend_yield': 0.4,
  '52_week_high': 260.1,
  '52_week_low': 169.21,
  'beta': 1.094,
  'sector': 'Technology',
  'industry': 'Consumer Electronics',
  'company_summary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App S